In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define paths
zip_path = "/content/drive/My Drive/HouseGAN/dataset_paper.zip"  # CHANGE THIS to match your file
extract_path = "/content/dataset"  # Where to extract in Colab

# Extract ZIP
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully!")

In [ ]:
!ls /content/dataset

In [ ]:
import pandas as pd

csv_path = "/content/dataset/floorplans.csv"  # Modify this
df = pd.read_csv(csv_path)

print(df.head())  # Show first few rows

In [ ]:
import glob

image_files = glob.glob("/content/dataset/images/*.png")  # Adjust path based on dataset
print(f"Found {len(image_files)} images.")

In [2]:
!git clone https://github.com/pmh47/House-GAN.git
%cd House-GAN
!pip install torch torchvision networkx matplotlib numpy

Cloning into 'House-GAN'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'House-GAN'
/content


In [12]:
import pandas as pd
dataset_file = "/content/drive/My Drive/datasets/floorplans.csv"  # Modify path
df = pd.read_csv(dataset_file)
print(df.head())

mkdir: cannot create directory ‘pretrained’: File exists
--2025-03-22 05:39:36--  https://github.com/pmh47/House-GAN/releases/download/v1.0/housegan.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-22 05:39:37 ERROR 404: Not Found.



In [ ]:
import networkx as nx

def create_graph_from_data(data):
    G = nx.Graph()

    for idx, row in data.iterrows():
        G.add_node(idx, label=row["room_type"])  # Example: Assign room type

    for idx in range(len(data) - 1):
        G.add_edge(idx, idx + 1)

    return G

G = create_graph_from_data(df)


In [ ]:
import torch
from models import HouseGAN

device = "cuda" if torch.cuda.is_available() else "cpu"
model = HouseGAN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

for epoch in range(10):
    optimizer.zero_grad()
    output = model(G)
    loss = loss_fn(output, target)  # Define `target` based on dataset
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item()}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = HouseGAN().to(device)
model.load_state_dict(torch.load("pretrained/housegan.pth", map_location=device))
model.eval()  # Set to evaluation mode

In [ ]:
with torch.no_grad():
    floorplan = model.sample()  # Generate a new floorplan

# Convert to a NetworkX graph for visualization
G = nx.Graph()
for room in floorplan["rooms"]:
    G.add_node(room["id"], label=room["type"])
for connection in floorplan["connections"]:
    G.add_edge(connection["room1"], connection["room2"])

# Draw the floorplan graph
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G)  # Layout for better visualization
nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray", node_size=2000, font_size=10)
plt.title("Generated Floorplan")
plt.show()

In [ ]:
plt.savefig("generated_floorplan.png")
from google.colab import files
files.download("generated_floorplan.png")